# Lab 10: Modeling Gravitational Motion

_Group Members_ :

In this lab, you will simulate several gravitational orbits, starting with the Earth orbiting the Sun. You will employ similar code to that used in previous computational labs. Hopefully the _method_ for simulating will be familiar, allowing you to focus on the impact of initial conditions and simulation parameters. With this in mind, we have outlined much of the simulation for you already.

# Part One: Simulating Earth's Orbit

In this section, we will simulate the Earth in orbit around the Sun. This will follow a similar approach as in previous computational labs: we simply need to calculate the force acting on an object moving in two dimensions and update its acceleration, velocity, and position at discrete time steps. This same general approach allows us to model a wide variety of systems, by simply defining the appropriate force function.

To model a planet orbiting a sun, the relevant force is given by _Newton's law of universal gravitation._ This gives the gravitational force $\vec F_{12}$ acting on mass $m_1$ due to mass $m_2$ and takes the form

\begin{equation}
\large \vec F_{12} = -G \dfrac{m_1 m_2}{r^2} \hat{r},
\end{equation}

where $r$ is the distance between $m_1$ and $m_2$, $\hat r$ is a unit vector pointing from $m_2$ to $m_1$, and $G$ is the _gravitational constant_.

**<font color=A07A06>Problem 10.1: In previous computational labs, your force function could depend on various parameters, such as position and velocity. To model the Earth orbiting the Sun, what parameters should your force function depend on?**

_Double click this cell to begin editing. Write your answer here._

We will begin with some code that looks similar to what you produced in Lab 6. First we import the relevant libraries:

**<font color=grey>Code Task 10.1: Run the cell below. <font color=red>You may use FFMPEG for this lab if you wish, but the animations in this lab are optional. If you don't want to spend time on them but would still like to visualize or see the motion, discuss the relevant questions with your instructor. To use FFMPEG, uncomment the bottom two lines after you've specified the FFMPEG path.**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML


'''
ffmpeg: for Anaconda installations, we can open an Anaconda prompt and type 'conda install ffmpeg'
to install ffmpeg. But since there are issues with the setup on the lab computers, we can manually
point to the ffmpeg binary (as below). Eventually, we can remove the lines that follow.
For now, we need to change the path below to point to the ffmpeg binary (wherever you choose to put it).
Windows users can get the ffmpeg binary at https://ffmpeg.zeranoe.com/builds/.
'''
#rc('animation', html='html5')  # this lets us call the animation object directly, without having to explicitly compile it

#plt.rcParams['animation.ffmpeg_path'] = r'???'

Next, we define a _class_ called ``ball``. We will create planets as instances of the ``ball`` class. Each instance of the ``ball`` class will have variables ``r``, ``v``, and ``a``, for instantaneous values of position, velocity, and acceleration. Likewise, the arrays ``r_array``, ``v_array``, and ``a_array`` store the _history_ of position, velocity, and acceleration.

**<font color=A07A06>Code Task 10.2: Run the cell below.**

In [ ]:
class ball:
    
    def __init__(self, m=1, r=np.array([0,0,0]), v=np.array([0,0,0]), a=np.array([0,0,0])):
        
        self.m = m
        
        self.r = r
        self.v = v
        self.a = a
        
        self.r_array = np.array([])
        self.v_array = np.array([])
        self.a_array = np.array([])

Now we are ready to define our force function. When complete, this function will take two objects of the ``ball`` class (such as a planet and a sun) as input, and will return the force between them as output.

**<font color=A07A06>Code Task 10.3: Complete the force function below by entering Newton's law of universal gravitation in the location marked by ``# YOUR CODE HERE``. Then run the cell.**

Hints:
1. You can use the variables ``G``, ``ball_1.m``, ``ball_2.m``, ``r``, and ``r_hat``.
1. You can calculate the dot product of two vectors ``a`` and ``b`` as ``a.dot(b)``.

In [ ]:
# Gravitational constant

G = 6.674e-11  # N m^2 / kg^2

# Force function: inputs are ball_1 and ball_2

def force(ball_1, ball_2):
    '''
    This is the force that ball_1 feels due to ball_2. This is dependent
    only on the position and mass of the balls, and the gravitational constant (G).
    '''
    output_force = np.zeros(3)  # Initialze the returned force array with zeros
    
    r = ball_1.r - ball_2.r  # Position vector between ball_2 and ball_1
    
    # Quick check if r^2 is zero to avoid division by zero
    if np.sqrt(r.dot(r)) == 0:
        return output_force
    
    r_hat = r / np.sqrt(r.dot(r))  # Unit vector pointing from ball_2 to ball_1
    
    # YOUR CODE HERE
    output_force = ???
    
    return output_force

Next up, we need to initialize our system. We create ``earth`` and ``sun`` objects as instances of the ``ball`` class. We need to give the ``earth`` and ``sun`` properties such as mass, initial position, and initial velocity. We will place the ``sun`` at the origin and the ``earth`` on the positive $x$-axis, separated from the ``sun`` by a distance equal to the Earth's mean orbital radius. For now, assume the initial velocity of the ``sun`` is zero. Let's make the initial velocity of the ``earth`` point in the $+\hat y$ direction at some speed you get to choose! **To get you in the right ballpark, we recommend choosing an initial velocity between 12,000 m/s and 25,000 m/s.**

**<font color=A07A06>Code Task 10.4: Choose an initial velocity for the Earth and enter it in the cell below. Then run the cell.**

In [ ]:
v_initial_earth =  ??? # Your answer here

# Physical constants: mass of the Sun, mass of the Earth, and average distance between the two

M_Sun = 1.988e30  # kg
M_Earth = 5.972e24  # kg
R_Earth = 1.496e11  # m

# Create instances of the ball class for the earth and sun

earth = ball(m=M_Earth, r=np.array([R_Earth,0,0]), 
             v=np.array([0, v_initial_earth, 0]))
sun = ball(m=M_Sun)

Now we need to split up time into discrete steps. We will simulate events happening over a total time ``T``, using time steps of size ``dt``.

**<font color=A07A06>Problem 10.2: What do you think would be appropriate values for ``T`` and ``dt``? Explain why you chose these particular values, and also note the values you chose.**

Hint: A year is about $3 \times 10^7$ s.

_Double click this cell to begin editing. Write your answer here._

**<font color=A07A06>Code Task 10.5: Enter your values for ``T`` and ``dt`` in the cell below. Then run the cell.**

In [ ]:
T = ???# Your answer here
dt = ???# Your answer here

times = np.arange(0, T+dt, dt)

N = times.size

earth.a_array = np.empty((N,3))
earth.a_array[0] = earth.a
earth.v_array = np.empty((N,3))
earth.v_array[0] = earth.v
earth.r_array = np.empty((N,3))
earth.r_array[0] = earth.r

sun.a_array = np.empty((N,3))
sun.a_array[0] = sun.a
sun.v_array = np.empty((N,3))
sun.v_array[0] = sun.v
sun.r_array = np.empty((N,3))
sun.r_array[0] = sun.r

**<font color=A07A06>Code Task 10.6: Run the cell below to simulate the orbit.**

In [ ]:
i = 1
for t in times[1:]:

    # first update the accelerations
    earth.a = force(earth, sun)/earth.m
    sun.a = 0
    # then append the new value to the list
    earth.a_array[i] = earth.a
    sun.a_array[i] = sun.a

    # second update the velocities
    earth.v = earth.v + earth.a * dt
    sun.v = sun.v + sun.a * dt
    # then append the new value to the list
    earth.v_array[i] = earth.v  
    sun.v_array[i] = sun.v  

    # third update the positions
    earth.r = earth.r + earth.v * dt
    sun.r = sun.r + sun.v * dt
    # then append the new value to the list
    earth.r_array[i] = earth.r
    sun.r_array[i] = sun.r

    # update iteration count
    i = i + 1

**<font color=A07A06>Code Task 10.7: Run the cell below to plot your simulated data points.**

In [ ]:
plt.plot(earth.r_array[:,0], earth.r_array[:,1], '.')
plt.plot(sun.r_array[:,0], sun.r_array[:,1], '.')
plt.axis('equal')
plt.show()

**<font color=A07A06>Problem 10.3: Look at the graph above and describe the orbit you have modeled. Is it circular? Elliptical? How does this differ from the Earth's orbit? Note that we chose realistic values for the masses and orbital radius, but not the Earth's initial velocity.**

_Double click this cell to begin editing. Write your answer here._

Take a look at the graph above, and reassess your choices of simulation parameters ``T`` and ``dt``. Does your Earth make at least one complete orbit? If not, you may need a larger ``T``. Does it trace the same path several times? Maybe make ``T`` smaller. Do the points on the planet's path seem too far apart? Or does the orbit not meet up in a closed loop? You may need to make ``dt`` smaller. Did the cells take too long to run, and the data points are so close together that the graph looks like a solid line? Maybe make ``dt`` larger.

Throughout this lab, we hope that you will ask yourself these questions to make sure you are optimizing your choices of ``T`` and ``dt`` for each simulation.

**<font color=A07A06>Code Task 10.8: Go back to Code Task 5 and try different values for ``T`` and ``dt`` until you are satisfied. (Be sure to run all cells between Code Task 3 and here to see your results.)**

Next, we will animate the orbit by rendering a video of all our simulated data points. (We do not expect you to dig into _how_ the animation code works.)

**<font color=grey>Code Task 10.9: Run the next two cells to animate your orbit! (The video will take a few moments to render.)**

In [ ]:
def init():
    line.set_data([], [])
    return(line)

def animate_ball_list(ball_list):
    def animation_function(i):
        x = []
        y = []
        for j in range(0,len(ball_list)):
            x.append(ball_list[j].r_array[i, 0])
            y.append(ball_list[j].r_array[i, 1])
        line.set_data(x, y)
        return (line)
    return animation_function

In [ ]:
fig3, axs3 = plt.subplots()
axs3.set_xlim((np.min(earth.r_array[:,0]) - 0.1*np.abs(np.min(earth.r_array[:,0])),
               np.max(earth.r_array[:,0]) + 0.1*np.abs(np.max(earth.r_array[:,0]))))
axs3.set_ylim((np.min(earth.r_array[:,1]) - 0.1*np.abs(np.min(earth.r_array[:,1])),
               np.max(earth.r_array[:,1]) + 0.1*np.abs(np.max(earth.r_array[:,1])))
             )
axs3.set_aspect('equal')
axs3.plot([-1,1], [0,0], color='C1', lw=2)
line, = axs3.plot([], [], '.', markersize=24, lw=2)

plt.close()

anim_2 = animation.FuncAnimation(fig3, animate_ball_list([sun, earth]), init_func=init, frames=N, interval=40)
anim_2

<strong><font color=004D40>Problem 10.4:  For an elliptical orbit, discuss the velocity of the Earth when it is near the Sun (perihelion) versus far from the Sun (aphelion).  What do you notice? If you did not run the animation, you can find the index of the earth array at its max and min to find the velocity at those points using the code below.

_Double click this cell to begin editing. Write your answer here._

In [ ]:
rMag = np.sqrt(earth.r_array[:,0]**2 + earth.r_array[:,1]**2)
rMaxIndex = np.argmax(rMag)
rMinIndex = np.argmin(rMag)

vMag = np.sqrt(earth.v_array[:,0]**2 + earth.v_array[:,1]**2)

print('At its maximum distance, the speed of earth is {:.3f} m/s'.format(vMag[rMaxIndex]))
print('At its minimum distance, the speed of earth is {:.3f} m/s'.format(vMag[rMinIndex]))

<strong><font color=004D40> Problem 10.5: Why is the velocity larger or smaller at aphelion vs perihelion? Hint: Think about whether or not energy is conserved and what happens to the kinetic and potential energy at these points.

_Double click this cell to begin editing. Write your answer here._

# Part Two: Circular Orbit

You might be looking at the orbit above and wondering why it's so elliptical. If we actually lived on a planet in that orbit we would burn up once a year. The remedy lies in the initial conditions. Next we're going to play with the initial conditions to try and make a circular orbit. 

Playing with this means compressing a lot of the steps above so that we can see everything at once. For example, we can compress all the lines adjusting the planets' accelerations, velocities, and positions into an update function (much like how we compressed all the calculations for the gravitational force into a force function). Similarly, we want to move all the tedious lines of code setting initial conditions into an initialization function. 

**<font color=004D40>Code Task 10.10: Read and understand what is happening below. Then run the next two cells.**

In [ ]:
def initialize_planet(planet, N):
    '''
    This takes in a planet and initializes it with
    empty arrays for the position, velocity, and
    acceleration histories. N denotes the total number
    of time steps these histories will contain.
    '''
    
    planet.a_array = np.empty((N,3))
    planet.a_array[0] = planet.a
    planet.v_array = np.empty((N,3))
    planet.v_array[0] = planet.v
    planet.r_array = np.empty((N,3))
    planet.r_array[0] = planet.r

In [ ]:
def update_planet(planet, force_felt, dt):
    '''
    This takes in a planet and the force it felt and
    updates the planet by one time step.
    '''
    # first update the acceleration
    planet.a = force_felt/planet.m
    # then append the new value to the list
    planet.a_array[i] = planet.a
    # second update the velocities
    planet.v = planet.v + planet.a * dt
    # then append the new value to the list
    planet.v_array[i] = planet.v  
    # third update the positions
    planet.r = planet.r + planet.v * dt
    # then append the new value to the list
    planet.r_array[i] = planet.r

**<font color=004D40>Code Task 10.11: Change the velocity of the earth to get a roughly circular orbit (you could go up to 35,000 m/s, faster than suggested in part one). You can run this cell as many times as you need!**

Hint: Your orbit may not "look" circular, even when it is! Be sure to note the horizontal vs. vertical scaling of the graph.

In [ ]:
v_initial_earth = ??? # Your answer here

T = ???# Your answer here
dt = ???# Your answer here

times = np.arange(0, T+dt, dt)

N = times.size


earth = ball(m=M_Earth, r=np.array([R_Earth,0,0]), 
             v=np.array([0, v_initial_earth, 0]))
sun = ball(m=M_Sun)

initialize_planet(earth, N)
initialize_planet(sun, N)

i = 1
for t in times[1:]:
    
    # First find the force
    earth_force = force(earth, sun)
    sun_force = 0
    
    # Update the planet(s)
    update_planet(earth, earth_force, dt)
    update_planet(sun, sun_force, dt)
    
    i = i + 1

# Plotting

plt.plot(earth.r_array[:,0], earth.r_array[:,1], '.')
plt.plot(sun.r_array[:,0], sun.r_array[:,1], '.')
plt.axis('equal')
plt.show()

**<font color=004D40>Problem 10.6: What orbital velocity did you settle on to get a circular orbit? Look up the Earth's actual orbital velocity and see how close you were! (https://www.wolframalpha.com/input/?i=earth+orbital+velocity+in+m%2Fs)**

_Double click this cell to begin editing. Write your answer here._

# Part Three: Time to Play!

Now that we have some sense of how Earth's orbit works, and how changing our parameters changes the results, it is time to mess with our model. Before we begin, we should introduce some quantitative measure of our orbit to serve as evidence of the changes we introduce. One such measure is the orbital period (year), which we will calculate below.

**<font color=1E88E5>Code Task 10.12: Read and understand what is happening below. Then run the cell.**

In [ ]:

def length_of_year(planet, dt, tolerance=R_Earth*0.01):
    '''
    This takes in a planet after it has orbitted and calculates
    the length of the year based on how long it takes for it to
    return back to its starting point. Because our simulation
    is not perfect, some tolerance is allowed in case it misses
    its starting point by a little
    '''
    def distance_from_start(index):
        '''
        This calculates the distance at a given time the planet
        is from its start location
        '''
        start_loc = planet.r_array[0]
        curr_loc = planet.r_array[index]
        distance = curr_loc - start_loc
        return np.sqrt(distance.dot(distance))
    
    # This loop seems a little complicated, but really it is just first
    # checking that the planet got away from start and then came back,
    # so it doesn't just think it has finished a loop immediately
    near_start = True
    for i in range(len(planet.r_array)):
        if near_start and distance_from_start(i) > tolerance:
            near_start = False
        if not near_start and distance_from_start(i) < tolerance:
            return i*dt
    return 0  # If something went wrong, it will say the year was instant

Now, our first modification is going to be to make the sun feel force. Up to now, we have treated the Sun as completely static, which is rather artificial. Remember Newton's third law: the force on the Earth due to the sun is matched with an equal and opposite force on the Sun due to the Earth. We're going to account for the force on the Sun by treating the Sun as a really heavy planet. Then we can generalize our code a bit, treating the two objects on a truly equal footing.

In particular, we are going to change our code by looping over a list of "planets" (including the Sun) rather than listing out each one every time we update it, apply force, or initialize it.

**<font color=1E88E5>Code Task 10.13: Read and understand what is happening below. (Note how we are making our code more efficient and compact as we go.) Then run the cell.**

In [ ]:
v_initial_earth = 29800  # m/s

earth = ball(m=M_Earth, r=np.array([R_Earth,0,0]), 
             v=np.array([0, v_initial_earth, 0]))
sun = ball(m=M_Sun)

planets = [earth, sun]

for planet in planets:
    initialize_planet(planet, N)

i = 1
for t in times[1:]:
    
    # first find the force
    for planet in planets:
        planet.force = 0
        for second_planet in planets:
            planet.force += force(planet, second_planet)
    
    # Update the planet(s)
    for planet in planets:
        update_planet(planet, planet.force, dt)
    
    i = i + 1

print(length_of_year(earth, dt))

plt.plot(earth.r_array[:,0], earth.r_array[:,1], '.')
plt.plot(sun.r_array[:,0], sun.r_array[:,1], '.')
plt.axis('equal')
plt.show()

Note the numerical output above the graph. This is the output of our ``length_of_year`` function (in seconds). Compare to the actual Earth year of $3.154 \times 10^7$ s.

**<font color=1E88E5>Problem 10.7: Compare your result from code task 13 to the actual length of an Earth year.**

_Double click this cell to begin editing. Write your answer here._

**<font color=1E88E5>Problem 10.8: We made our simulation account for the force on the Sun, but the Sun... still isn't moving. Why do you think this is? Is it realistic?**

_Double click this cell to begin editing. Write your answer here._

_Noowwwww,_ we are going to really mix things up. If we want to get the Sun moving, we need a larger force. So let's make the Earth more massive! Specifically, let's make the Earth have ``m=M_Sun``. We will also set up the Earth and Sun to begin with equal initial speeds (in the appropriate directions to begin an orbit).

**<font color=1E88E5>Code Task 10.14: Run the cell below, and compare the output to what you got before. (They should be identical.) Then, update the Earth's mass to ``m=M_Sun``, and update the Earth's initial velocity to ``[0, 15000, 0]`` and the Sun's initial velocity to ``[0, -15000, 0]``. Then run the cell again.**

In [ ]:
earth = ball(m=M_Earth, r=np.array([R_Earth,0,0]), 
             v=np.array([0, 29800, 0]))
sun = ball(m=M_Sun, 
             v=np.array([0, 0, 0]))

planets = [earth, sun]

for planet in planets:
    initialize_planet(planet, N)

i = 1
for t in times[1:]:
    
    # first find the force
    for planet in planets:
        planet.force = 0
        for second_planet in planets:
            planet.force += force(planet, second_planet)
    
    # Update the planet(s)
    for planet in planets:
        update_planet(planet, planet.force, dt)
    
    i = i + 1

print(length_of_year(earth, dt))

plt.plot(earth.r_array[:,0], earth.r_array[:,1], '.')
plt.plot(sun.r_array[:,0], sun.r_array[:,1], '.')
plt.axis('equal')
plt.show()

Now that we have made the Earth and Sun have equal masses, the Sun's motion becomes significant, and we see the two bodies orbiting each other!

**<font color=1E88E5>Problem 10.9: Discuss what you did in code task 14, and the result of that.  What physical system could this represent?**

_Double click this cell to begin editing. Write your answer here._

# Cumulative Assessment Task (CAT)

For the Culminating Assessment Task (CAT), you will simulate a three-body system. Specifically, the system should have two bodies, both with mass ``M_Sun``, orbiting each other (like the system above). A third body, with mass ``M_Earth``, should closely orbit one of the first two bodies.

**<font color=D81B60>Code Task 10.15: We have chosen a starting point for ``T`` and ``dt``. Run the cell below.**

In [ ]:
T = 4e7
dt = 1e5

We have chosen the following initial conditions to get you started:
- Body #1:
    - Mass: ``M_Sun``
    - Initial position: a distance ``R_Earth`` to the left of the origin
    - Initial velocity: 9,000 m/s in the $-\hat y$ direction
- Body #2:
    - Mass: ``M_Sun``
    - Initial position: a distance ``R_Earth`` to the right of the origin
    - Initial velocity: 9,000 m/s in the $+\hat y$ direction
- Body #3:
    - Mass: ``M_Earth``
    - Initial position: a distance ``0.9*R_Earth`` to the left of the origin (just to the right of Body #1)
    - Initial velocity: 100,000 m/s in the $+\hat y$ direction

**<font color=D81B60>Code Task 10.16: Update the code below to create a list of three balls using the initial conditions given above. Then run the cell.**

In [ ]:
suns = [ball(m=0, r=np.array([0, 0, 0]), v=np.array([0, 0, 0])),
        ball(m=0, r=np.array([0, 0, 0]), v=np.array([0, 0, 0])),
        ball(m=0, r=np.array([0, 0, 0]), v=np.array([0, 0, 0]))]

**<font color=D81B60>Code Task 10.17: Run the cell below to simulate the three-body orbit!**

In [ ]:
times = np.arange(0, T+dt, dt)
N = times.size

for sun in suns:
    initialize_planet(sun, N)

i = 1
for t in times[1:]:
    
    # first find the force
    for planet in suns:
        planet.force = 0
        for second_planet in suns:
            planet.force += force(planet, second_planet)
    
    # Update the planet(s)
    for planet in suns:
        update_planet(planet, planet.force, dt)
    
    i = i + 1

print(length_of_year(suns[2], dt, tolerance=R_Earth*0.1))
for sun in suns:
    plt.plot(sun.r_array[:,0], sun.r_array[:,1], '.')
plt.xlim(-1.5*R_Earth, 1.5*R_Earth)
plt.ylim(-R_Earth, R_Earth)
plt.axis('equal')
plt.show()

That's pretty neat! If everything has gone well, you should see that the two massive bodies (blue and yellow) make complete ellipses. Meanwhile, the green planet tightly orbits the blue body, but eventually rockets off into the distance.

Simulation is a powerful tool. Today, we have simulated systems that we could never fit in a lab. This allowed us to get a "hands-on" experience with gravitational systems in a way that would not be possible without simulation. However, we need to be careful. In this case, due to the parameters we gave you, the simulation does not give us a physically accurate result. Given the masses, initial positions, and initial velocities, it turns out the green planet should _not_ actually rocket off into the distance. Instead, it should continue orbiting the blue planet.

Notice how the simulated points for the green planet are spaced a somewhat far apart. Given that this planet is moving quite fast, perhaps we have chosen too large a value for the time step ``dt``.

**<font color=D81B60>Code Task 10.18: Go back to Code Task 15 and try different values for ``dt`` until you are satisfied. (Be sure to run all cells between Code Task 15 and here to see your results.)**

**<font color=D81B60>Problem 10.10: How can you feel confident that your simulation is showing a physically accurate result, as opposed to an artifact of the simulation parameters? How do you know if you chose a "good" value for ``dt``?**

_Double click this cell to begin editing. Write your answer here._

**<font color=D81B60>Problem 10.11: What physical system could this represent?**

_Double click this cell to begin editing. Write your answer here._

**<font color=grey>Code Task 10.19: Once you are happy with your simulation parameters, run the cell below to animate your simulation! (It may take several minutes to render the video.)**

In [ ]:
fig, axs = plt.subplots()
xMin = 0
xMax = 0
yMin = 0
yMax = 0
for sun in suns:
    xMin = min(xMin, np.min(sun.r_array[:,0]))
    yMin = min(yMin, np.min(sun.r_array[:,1]))
    xMax = max(xMax, np.max(sun.r_array[:,0]))
    yMax = max(yMax, np.max(sun.r_array[:,1]))
axs.set_xlim(xMin, xMax)
axs.set_ylim(yMin, yMax)
axs.set_aspect('equal')
axs.plot([-1,1], [0,0], color='C1', lw=2)
line, = axs.plot([], [], '.', markersize=24, lw=2)
plt.close()

anim_many = animation.FuncAnimation(fig, animate_ball_list(suns), init_func=init, frames=N, interval=1)
anim_many

### <center>You're done! Submit your lab per the following instructions.<font color=white><#!S!#>

* Save this file with the lab number and group number to submit to canvas, e.g. `group01_lab02.ipynb`. 
* You only need to submit one file as a group to the canvas assignment and it will count for all group members.
* The group recorder should still send the worksheet to their group members so they have a copy.
* Save the file as a html file. You can do this by opening the `File` tab on the top left corner of Jupyter, then select `Download as > html (.html)`
* Submit both the .ipynb and .html files to canvas under the `Lab 10: Modeling Gravitational Motion` assignment.